# Transit Matrix

In [2]:
import sys, os
os.chdir('/home/sihan/UChicago/Projects/spatial_access/spatial_access')
from p2p import *
from spatial_access.Models import *
%matplotlib inline

**View data structure: Schools in Beijing.**  

In [9]:
df = pd.read_csv('data/dest/school.csv')

In [10]:
df.rename(columns={'KIND':'cate_code'}, inplace=True)

In [11]:
df['OBJECTID'] = df['OBJECTID'].apply(str).str[:-2]

In [12]:
dict_cate = {
"A700": "Universities",
"A701": "Kindergarten",
"A702": "Primary Schools",
"A703": "High Schools",
"A708": "Occupational Schools",
"A709": "Occupational Schools"
}

In [13]:
df['category'] = df['cate_code'].map(dict_cate)

In [14]:
df.head()

,id,cate_code,OBJECTID,lon,lat,category
0,1,A701,3,116.529609,40.096216,Kindergarten
1,2,A703,4,116.653038,39.851399,High Schools
2,3,A701,2424,116.826920,40.370471,Kindergarten
3,4,A702,2424,116.955374,40.561026,Primary Schools
4,5,A703,5,116.456567,40.048734,High Schools


In [15]:
df.groupby('cate_code').count()

,id,OBJECTID,lon,lat,category
cate_code,,,,,
A700,661,661,661,661,661
A701,1540,1540,1540,1540,1540
A702,1390,1390,1390,1390,1390
A703,772,772,772,772,772
A706,110,110,110,110,0
A708,301,301,301,301,301
A709,1414,1414,1414,1414,1414


In [16]:
df_model = df[df.category.notnull()]

In [17]:
df_model.to_csv('data/dest/school_model.csv', encoding='utf-8', index=False)

**View data structure: Hospitals in Beijing.**  

In [8]:
dfh = pd.read_csv('data/dest/hospital_clean.csv')

In [23]:
dfh.head()

,id,cate_code,lon,lat,category
0,1,7280,117.260982,40.164635,Clinics
1,2,7280,117.261252,40.131678,Clinics
2,3,7280,116.634666,40.321699,Clinics
3,4,7280,116.776288,39.775103,Clinics
4,5,7280,116.676865,40.280147,Clinics


In [11]:
dfh.rename(columns={'kind':'cate_code'}, inplace=True)

In [25]:
dfh.groupby('cate_code').count()

,id,lon,lat,category
cate_code,,,,
7200,850,850,850,850
7204,445,445,445,445
7205,159,159,159,159
7280,2865,2865,2865,2865
7500,61,61,61,61
7580,269,269,269,269


In [17]:
dfh['cate_code'] = dfh['cate_code'].apply(str)

In [20]:
dict_hcate = {
"7280": "Clinics",
"7200": "Hospitals",
"7204": "Outpatient Facilities",
"7580": "Animal Hospitals",
"7500": "Disease Control and Prevention Centers",
"7205": "Emergency Departments"
}

In [22]:
dfh['category'] = dfh['cate_code'].map(dict_hcate)

In [26]:
dfh_model = dfh[dfh.category.notnull()]

In [27]:
dfh_model.to_csv('data/dest/hospital_model.csv', encoding='utf-8', index=False)

**View data structure: Subway Stations in Beijing.**  

In [53]:
dfs = pd.read_csv('data/dest/subway.csv')

In [54]:
dfs['category'] = "subway"

In [55]:
dfs.head()

,lon,lat,id,category
0,116.309893,39.872521,1,subway
1,116.300326,39.872593,2,subway
2,116.281826,39.871571,3,subway
3,116.259835,39.870755,4,subway
4,116.246953,39.865215,5,subway


In [57]:
dfs.to_csv('data/dest/subway_model.csv', encoding='utf-8', index=False)

***

## 1. Asymmetric Distance Matrix

### 1.1 School Matrix by Walking

In [45]:
# Calculate asymmetric distance matrix for walking (takes ~5 minutes to run) 
w_asym_mat = TransitMatrix(network_type='walk',
                           primary_input='data/orig/origin.csv',
                           secondary_input='data/dest/school_model.csv',
                           configs=Configs(disable_area_threshold=True, 
                                           epsilon=0.05))

In [46]:
w_asym_mat.process()
#The output is walk_asym_health_tracts.csv (used in the calculation of the metrics)

The variables in your data set are:
>  id
>  lat
>  lon
>  type
>  OBJECTID
Enter the longitude coordinate: lon
Enter the latitude coordinate: lat
Enter the index name: id
The variables in your data set are:
>  id
>  cate_code
>  OBJECTID
>  lon
>  lat
>  category
Enter the longitude coordinate: lon
Enter the latitude coordinate: lat
Enter the index name: id


INFO:p2p:All operations completed in 19.02 seconds


In [30]:
w_asym_mat.write_csv('/mnt/d/Auguste/Projects/soh-bj/data/walk_school_ep0.05.csv')

INFO:p2p:Wrote to /mnt/d/Auguste/Projects/soh-bj/data/walk_school_ep0.05_1.csv in 0.24 seconds


In [47]:
w_asym_mat.write_tmx('/mnt/d/Auguste/Projects/soh-bj/data/walk_school_ep0.05.tmx')

INFO:p2p:Wrote to /mnt/d/Auguste/Projects/soh-bj/data/drive_school_ep0.05.tmx in 0.12 seconds


### 1.2 Hospital Matrix by Walking

In [29]:
# Calculate asymmetric distance matrix for walking (takes ~5 minutes to run) 
w_asym_mat_h = TransitMatrix(network_type='walk',
                           primary_input='data/orig/origin.csv',
                           secondary_input='data/dest/hospital_model.csv',
                           configs=Configs(disable_area_threshold=True, 
                                           epsilon=0.05))

In [30]:
w_asym_mat_h.process()

The variables in your data set are:
>  id
>  lat
>  lon
>  type
>  OBJECTID
Enter the longitude coordinate: lon
Enter the latitude coordinate: lat
Enter the index name: id
The variables in your data set are:
>  id
>  cate_code
>  lon
>  lat
>  category
Enter the longitude coordinate: lon
Enter the latitude coordinate: lat
Enter the index name: id
Requesting network data within bounding box from Overpass API in 16 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](39.41318600,116.97178289,39.81247549,117.46926900);>;);out;'}"
Downloaded 2,116.9KB from www.overpass-api.de in 2.33 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!

/home/sihan/.local/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['area', 'hgv', 'highway', 'junction', 'maxspeed', 'name', 'oneway', 'ref', 'toll', 'width']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
INFO:p2p:Finished querying osm
INFO:p2p:All operations completed in 185.73 seconds


In [31]:
w_asym_mat_h.write_csv('/mnt/d/Auguste/Projects/soh-bj/data/walk_hospital_ep0.05.csv')

INFO:p2p:Wrote to /mnt/d/Auguste/Projects/soh-bj/data/walk_hospital_ep0.05.csv in 0.11 seconds


In [32]:
w_asym_mat_h.write_tmx('/mnt/d/Auguste/Projects/soh-bj/data/walk_hospital_ep0.05.tmx')

INFO:p2p:Wrote to /mnt/d/Auguste/Projects/soh-bj/data/walk_hospital_ep0.05.tmx in 0.03 seconds


### 1.3 Subway Matrix by Walking

In [46]:
w_asym_mat_s = TransitMatrix(network_type='walk',
                           primary_input='data/orig/origin.csv',
                           secondary_input='data/dest/subway.csv',
                           configs=Configs(disable_area_threshold=True, 
                                           epsilon=0.05))

In [47]:
w_asym_mat_s.process()

The variables in your data set are:
>  id
>  lat
>  lon
>  type
>  OBJECTID
Enter the longitude coordinate: lon
Enter the latitude coordinate: lat
Enter the index name: id
The variables in your data set are:
>  lon
>  lat
>  id
Enter the longitude coordinate: lon
Enter the latitude coordinate: lat
Enter the index name: id
Requesting network data within bounding box from Overpass API in 9 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](39.58804320,116.86405351,39.90512631,117.33874080);>;);out;'}"
Downloaded 1,458.9KB from www.overpass-api.de in 2.12 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](39.58997728,116.387

INFO:p2p:Finished querying osm
INFO:p2p:All operations completed in 158.23 seconds


In [48]:
w_asym_mat_s.write_tmx('/mnt/d/Auguste/Projects/soh-bj/data/walk_subway_ep0.05.tmx')

INFO:p2p:Wrote to /mnt/d/Auguste/Projects/soh-bj/data/walk_subway_ep0.05.tmx in 0.04 seconds


## 2. Access Score Model

### 2.1 Schools

In [58]:
accesst = AccessModel(network_type='walk',
                         transit_matrix_filename='/mnt/d/Auguste/Projects/soh-bj/data/walk_school_ep0.05.tmx',
                         sources_filename='data/orig/origin.csv',
                         source_column_names={'idx': 'id', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                         dest_column_names={'idx': 'id', 'category': 'category', 'capacity': 'skip', 'lat': 'lat', 'lon': 'lon'},
                         destinations_filename='data/dest/school_model.csv',
                         decay_function='logit')

In [63]:
dict_edu1 = {
"Universities": [5,4,3,2,1,1,1,1,1,1],
"High Schools": [10,10,9,9,8,8,7,7,6,6],
"Primary Schools": [10,10,9,9,8,8,7,7,6,6],
"Kindergarten": [10,10,9,9,8,8,7,7,6,6],
"Occupational Schools": [5,4,3,2,1,1,1,1,1,1]
}

In [64]:
accesst.calculate(upper_threshold=1800, category_weight_dict=dict_edu1, normalize=True)

INFO:spatial_access.BaseModel:Using weights: {'High Schools': [10, 10, 9, 9, 8, 8, 7, 7, 6, 6], 'Primary Schools': [10, 10, 9, 9, 8, 8, 7, 7, 6, 6], 'Occupational Schools': [5, 4, 3, 2, 1, 1, 1, 1, 1, 1], 'Kindergarten': [10, 10, 9, 9, 8, 8, 7, 7, 6, 6], 'Universities': [5, 4, 3, 2, 1, 1, 1, 1, 1, 1]}


,all_categories_score,High Schools_score,Primary Schools_score,Occupational Schools_score,Kindergarten_score,Universities_score
1,0.251615,0.166101,0.638266,0.119011,0.168919,0.077511
2,0.179373,0.000000,0.485082,0.057517,0.269042,0.000000
3,0.189387,0.048973,0.643986,0.011678,0.125002,0.000000
4,0.433480,0.354083,0.657475,0.598055,0.325226,0.518374
5,0.987937,0.998613,0.999958,0.999958,0.999975,0.848555
6,0.996205,0.996657,0.999888,0.999593,0.999824,0.965077
7,0.364047,0.347417,0.673600,0.010819,0.347515,0.084025
8,0.831308,0.885544,0.983116,0.884153,0.796702,0.252468
9,0.377086,0.214815,0.594013,0.551372,0.431082,0.187724
10,0.113450,0.007783,0.314975,0.238540,0.124942,0.102746


In [65]:
accesst.write_results('/mnt/d/Auguste/Projects/soh-bj/data/access_walk_school.csv')

### 2.2 Hospitals

In [35]:
accessth = AccessModel(network_type='walk',
                         transit_matrix_filename='/mnt/d/Auguste/Projects/soh-bj/data/walk_hospital_ep0.05.tmx',
                         sources_filename='data/orig/origin.csv',
                         source_column_names={'idx': 'id', 'population': 'skip', 'lat': 'lat', 'lon': 'lon'},
                         dest_column_names={'idx': 'id', 'category': 'category', 'capacity': 'skip', 'lat': 'lat', 'lon': 'lon'},
                         destinations_filename='data/dest/hospital_model.csv',
                         decay_function='logit')

In [40]:
dict_hos = {
"Clinics": [8,8,7,7,6,6,5,5,4,4],
"Hospitals": [10,10,10,9,9,9,8,8,8,7],
"Outpatient Facilities": [10,10,10,9,9,9,8,8,8,7],
"Emergency Departments": [8,8,7,7,6,6,5,5,4,4],
"Disease Control and Prevention Centers": [5,4,3,2,1,1,1,1,1,1]
}

In [41]:
accessth.calculate(upper_threshold=1800, category_weight_dict=dict_hos, normalize=True)

INFO:spatial_access.BaseModel:Using weights: {'Disease Control and Prevention Centers': [5, 4, 3, 2, 1, 1, 1, 1, 1, 1], 'Hospitals': [10, 10, 10, 9, 9, 9, 8, 8, 8, 7], 'Emergency Departments': [8, 8, 7, 7, 6, 6, 5, 5, 4, 4], 'Outpatient Facilities': [10, 10, 10, 9, 9, 9, 8, 8, 8, 7], 'Animal Hospitals': 'No decay', 'Clinics': [8, 8, 7, 7, 6, 6, 5, 5, 4, 4]}


,all_categories_score,Disease Control and Prevention Centers_score,Hospitals_score,Emergency Departments_score,Outpatient Facilities_score,Animal Hospitals_score,Clinics_score
1,0.185481,0.000000,0.171048,0.000000,0.000000,0.000000,0.740650
2,0.035686,0.000000,0.004122,0.000000,0.000000,0.000000,0.190713
3,0.142945,0.000000,0.000000,0.000000,0.000000,0.000000,0.791587
4,0.183553,0.000000,0.113657,0.000000,0.000000,0.000000,0.826095
5,0.846597,0.000000,0.998947,0.808499,0.647096,0.560341,0.999998
6,0.592121,0.022005,0.994427,0.245006,0.163957,0.388437,0.999971
7,0.170038,0.000000,0.282065,0.000000,0.000000,0.000000,0.469182
8,0.481136,0.556641,0.536589,0.188150,0.260894,0.106432,0.999915
9,0.156729,0.000000,0.105116,0.000000,0.000000,0.000000,0.691860
10,0.169649,0.000000,0.326835,0.000000,0.004701,0.002412,0.383595


In [42]:
accessth.write_results('/mnt/d/Auguste/Projects/soh-bj/data/access_walk_hospital.csv')

### 2.3 Subway

In [58]:
accesssub = AccessTime(network_type='walk',
                         transit_matrix_filename='/mnt/d/Auguste/Projects/soh-bj/data/walk_subway_ep0.05.tmx',
                         sources_filename='data/orig/origin.csv',
                         destinations_filename='data/dest/subway_model.csv')

The variables in your data set are:
>  id
>  lat
>  lon
>  type
>  OBJECTID
Enter the unique index variable: id
If you have no population variable, write "skip" (no quotations)
Enter the population variable: skip
Enter the latitude variable: lat
Enter the longitude variable: lon
The variables in your data set are:
>  lon
>  lat
>  id
>  category
Enter the unique index variable: id
If you have no capacity variable, write "skip" (no quotations)
Enter the capacity variable: skip
If you have no category variable, write "skip" (no quotations)
Enter the category variable: category
Enter the latitude variable: lat
Enter the longitude variable: lon


In [59]:
accesssub.calculate()

,time_to_nearest_subway,time_to_nearest_all_categories
1,2206,2206
2,3096,3096
3,3352,3352
4,2502,2502
5,104,104
6,8,8
7,2230,2230
8,5476,5476
9,2172,2172
10,8024,8024


In [60]:
dfs_result = accesssub.calculate()

In [84]:
dfs_result["normalized_score"] = (dfs_result["time_to_nearest_subway"].max() - dfs_result["time_to_nearest_subway"])/ (dfs_result["time_to_nearest_subway"].max()-dfs_result["time_to_nearest_subway"].min())

In [86]:
dfs_result.head()

,time_to_nearest_subway,time_to_nearest_all_categories,normalized_score
1,2206,2206,0.915127
2,3096,3096,0.880792
3,3352,3352,0.870915
4,2502,2502,0.903707
5,104,104,0.996219


In [89]:
dfs_result.to_csv('/mnt/d/Auguste/Projects/soh-bj/data/walk_nearest_subway.csv', encoding='utf-8', index=False)

## 3. Output

In [4]:
dfschool = pd.read_csv('/mnt/d/Auguste/Projects/soh-bj/data/access_walk_school.csv')

In [29]:
dfschool.rename(columns = {'all_categories_score':'school_score'}, inplace = True)

In [33]:
dfschool = dfschool.iloc[:,0:2]

In [6]:
dfsubway = pd.read_csv('/mnt/d/Auguste/Projects/soh-bj/data/walk_nearest_subway.csv')

In [27]:
dfsubway = dfsubway.iloc[:,0:2]

In [9]:
dfaod = pd.read_csv('/mnt/d/Auguste/Projects/soh-bj/data/monthly_aod.csv')

In [12]:
dfhospital = pd.read_csv('/mnt/d/Auguste/Projects/soh-bj/data/access_walk_hospital.csv')

In [22]:
dfhospital.rename(columns = {'all_categories_score':'hospital_score'}, inplace = True)

In [35]:
dfh = dfhospital.iloc[:,0:2]

In [42]:
df_merge = [dfaod, dfh,dfschool, dfsubway]

In [43]:
df_merge = [df.set_index('id') for df in df_merge]

In [47]:
df_merge = df_merge[0].join(df_merge[1:])

In [50]:
df_merge.head()

,lat,lon,type,jan_mean,feb_mean,mar_mean,apr_mean,may_mean,jun_mean,jul_mean,aug_mean,sep_mean,oct_mean,nov_mean,dec_mean,annual_avg,hospital_score,school_score,time_to_nearest_subway
id,,,,,,,,,,,,,,,,,,,
1,40.091667,116.236300,soh,NaN,0.386505,0.863163,0.865420,0.176904,NaN,0.171427,NaN,0.084822,NaN,0.103501,0.120693,0.346554,0.185481,0.251615,2206
2,40.107715,116.435483,soh,NaN,0.375618,0.815283,0.266202,0.195898,NaN,0.208812,NaN,0.108330,NaN,0.117074,0.118710,0.275741,0.035686,0.179373,3096
3,40.174420,116.330348,soh,NaN,0.394961,0.820186,0.385149,0.203651,NaN,0.166548,NaN,0.121637,NaN,0.121168,0.125720,0.292378,0.142945,0.189387,3352
4,40.227015,116.189257,soh,NaN,0.317308,0.870615,1.234435,0.143698,NaN,0.096689,NaN,0.057631,NaN,0.073201,0.098241,0.361477,0.183553,0.433480,2502
5,40.171901,117.167209,soh,0.365431,0.259009,0.715585,0.356510,0.218565,NaN,0.238579,0.496352,0.263193,0.231291,0.075490,0.101250,0.301932,0.846597,0.987937,104


In [51]:
df_merge.to_csv('/mnt/d/Auguste/Projects/soh-bj/data/result.csv', encoding='utf-8', index=False)